In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

In [18]:
cd /content/drive/MyDrive/workspace/vfnet

/content/drive/MyDrive/workspace/vfnet


In [ ]:
!tar -zxvf ./datafile/data.tar.gz -C /content

## Install

In [5]:
%cd /content

/content


In [6]:
!pip install torch==1.8.1+cu102 torchvision==0.9.1+cu102 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 804.1MB 21kB/s 
     |████████████████████████████████| 17.3MB 326kB/s 
     |████████████████████████████████| 1.9MB 14.5MB/s 
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
#!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.1/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.1/index.html
     |████████████████████████████████| 296kB 14.4MB/s 
     |████████████████████████████████| 194kB 25.0MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.4-cp37-cp37m-linux_x86_64.whl size=26350290 sha256=fa7b80bfcc781fe5cb2dd19590a1a9b1cf8b3338c214b5ec366df0311146c0cd
  Stored in directory: /root/.cache/pip/wheels/d4/39/64/573a0673de0ee0c6b699dddfae8232d485ea8da749caa198fd
Successfully built mmcv-full


In [7]:
!pip install mmcv-full==1.2.4 -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.1/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.1/index.html
     |████████████████████████████████| 225kB 14.8MB/s 
     |████████████████████████████████| 194kB 22.5MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.2.4-cp37-cp37m-linux_x86_64.whl size=21770343 sha256=fdec1a35fa009126589ec57fbc4fd82799bf303cf7a964c14b5607761db05664
  Stored in directory: /root/.cache/pip/wheels/dd/48/39/f64327bc1602fd0235b54b5a0f18e86bcc183ff2460c76376a
Successfully built mmcv-full


In [ ]:
%cd /content
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir ./

In [ ]:
!pip install timm
!pip install albumentations==0.4.6

In [10]:
%cd /content/drive/MyDrive/workspace/vfnet

/content/drive/MyDrive/workspace/vfnet


In [ ]:
!python setup.py develop
!pip install -r ./requirements.txt

In [12]:
!pip uninstall pycocotools -y

Uninstalling pycocotools-2.0.2:
  Successfully uninstalled pycocotools-2.0.2


In [ ]:
!pip install mmpycocotools==12.0.2

In [ ]:
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"

## mmdet

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import torch
import gc

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/custom_vfnet.py')

In [4]:
PREFIX = '/content/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'

'''
cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
'''
cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.data.samples_per_gpu = 4

cfg.seed=41
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/vfnet'

cfg.evaluation.metric = ['bbox']
cfg.log_config.interval = 300

#cfg.checkpoint_config = dict(type='CheckpointHook', interval=1)#, max_keep_ckpts=7)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [5]:
cfg.optimizer_config

{'grad_clip': {'max_norm': 35, 'norm_type': 2}}

In [ ]:
model = build_detector(cfg.model)

2021-05-18 05:44:34,204 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-18 05:44:34,444 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.convs.0.conv_offset.weight, layer2.0.convs.0.conv_offset.bias, layer2.0.convs.1.conv_offset.weight, layer2.0.convs.1.conv_offset.bias, layer2.0.convs.2.conv_offset.weight, layer2.0.convs.2.conv_offset.bias, layer2.1.convs.0.conv_offset.weight, layer2.1.convs.0.conv_offset.bias, layer2.1.convs.1.conv_offset.weight, layer2.1.convs.1.conv_offset.bias, layer2.1.convs.2.conv_offset.weight, layer2.1.convs.2.conv_offset.bias, layer2.2.convs.0.conv_offset.weight, layer2.2.convs.0.conv_offset.bias, layer2.2.convs.1.conv_offset.weight, layer2.2.convs.1.conv_offset.bias, layer2.2.convs.2.conv_offset.weight, layer2.2.convs.2.conv_offset.bias, layer2.3.convs.0.conv_offset.weight, layer2.3.convs.0.conv_offset.bia

In [ ]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=10.10s)
creating index...
index created!


In [ ]:
# /usr/local/lib/python3.7/dist-packages/mmcv/utils/path.py
# 36번줄 os.symlink 주석처리

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

## Inference

In [6]:
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
gc.collect()
torch.cuda.empty_cache()

In [7]:
file_name = 'best_bbox_mAP_pretrained.pth'

In [8]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{file_name}')

In [9]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [10]:
cfg.model.pretrained = None

In [ ]:
cfg.data.test

In [12]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

In [13]:
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [14]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 0.7 task/s, elapsed: 1126s, ETA:     0s

In [15]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'vfnet_single_tta_nms_fusion.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.1286355 136.8551 49.60831 255.36761 164.67...,batch_01_vt/0021.jpg
1,0 0.13566366 286.70135 333.85223 305.89117 376...,batch_01_vt/0028.jpg
2,0 0.1838766 234.20953 460.30057 302.0683 508.8...,batch_01_vt/0031.jpg
3,0 0.17768021 0.052265167 225.14793 32.6136 313...,batch_01_vt/0032.jpg
4,0 0.5369851 224.60435 391.9894 284.33554 458.8...,batch_01_vt/0070.jpg


In [16]:
submission.head(60)

,PredictionString,image_id
0,0 0.1286355 136.8551 49.60831 255.36761 164.67...,batch_01_vt/0021.jpg
1,0 0.13566366 286.70135 333.85223 305.89117 376...,batch_01_vt/0028.jpg
2,0 0.1838766 234.20953 460.30057 302.0683 508.8...,batch_01_vt/0031.jpg
3,0 0.17768021 0.052265167 225.14793 32.6136 313...,batch_01_vt/0032.jpg
4,0 0.5369851 224.60435 391.9894 284.33554 458.8...,batch_01_vt/0070.jpg
5,0 0.33754557 360.7737 225.246 372.65836 267.26...,batch_01_vt/0076.jpg
6,1 0.51500833 406.54605 225.20782 463.88843 271...,batch_01_vt/0078.jpg
7,0 0.3060061 27.410217 0.09117839 118.320274 12...,batch_01_vt/0096.jpg
8,0 0.28948668 272.27692 288.90823 430.23486 381...,batch_01_vt/0101.jpg
9,0 0.5687003 54.87259 0.15010262 419.68762 65.0...,batch_01_vt/0122.jpg
